# CODIV-19

In [1]:
import numpy as np
import pandas as pd
import torch
import pyro
import pyro.distributions as dist
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import pyro.poutine as poutine
from pyro.distributions import constraints

/home/matcraft/anaconda3/envs/pyro-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def sir_step(S, I, R, beta, gamma, N):
    dS = -beta * S * I / N
    dI = beta * S * I / N - gamma * I
    dR = gamma * I
    next_S = S + dS
    next_I = I + dI
    next_R = R + dR
    return next_S, next_I, next_R

In [3]:
# Define the SIR differential equations
def sir_model(y, t, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I
    dIdt = beta * S * I - gamma * I
    dRdt = gamma * I
    return [dSdt, dIdt, dRdt]

In [4]:
def sir_model_modified(susceptible, infected, recovered):
    # Prior distributions for the parameters
    beta = pyro.sample("beta", dist.Beta(2, 2))
    gamma = pyro.sample("gamma", dist.Beta(2, 2))
    
    # Initial conditions
    S, I, R = susceptible[0], infected[0], recovered[0]
    N = S + I + R

    # We'll use these lists to store our predictions at each time step
    S_pred = [S]
    I_pred = [I]
    R_pred = [R]
    
    # Loop over the observed data, predicting the next value using the previous value
    # and the ODE solver.
    for t in range(1, len(susceptible)):
        next_S, next_I, next_R = sir_step(S, I, R, beta, gamma, N)
        S_pred.append(next_S)
        I_pred.append(next_I)
        R_pred.append(next_R)
        
        S = next_S
        I = next_I
        R = next_R
        
    # Convert lists to tensors
    S_pred = torch.stack(S_pred)
    I_pred = torch.stack(I_pred)
    R_pred = torch.stack(R_pred)
    
    # Condition on the observed data
    pyro.sample("obs_S", dist.Normal(S_pred, 10), obs=susceptible)
    pyro.sample("obs_I", dist.Normal(I_pred, 10), obs=infected)
    pyro.sample("obs_R", dist.Normal(R_pred, 10), obs=recovered)

In [5]:
# Integrate the SIR equations over the time grid using given parameters
def sir_solution(y0, t, beta, gamma):
    # Ensure tensors are detached from the current graph
    t_np = t.detach().numpy()
    y0_np = [y.detach().numpy() for y in y0]
    beta_np = beta.detach().numpy()
    gamma_np = gamma.detach().numpy()
    return odeint(sir_model, y0_np, t_np, args=(beta_np, gamma_np))

In [6]:
# Define the Pyro model
def model(susceptible, infected, recovered):
    # Prior distributions for the parameters
    beta = pyro.sample("beta", dist.Uniform(0, 1))
    gamma = pyro.sample("gamma", dist.Uniform(0, 1))
    
    # Initial conditions: everyone susceptible, except for one infected individual
    S0 = susceptible[0]
    I0 = infected[0]
    R0 = recovered[0]

    # Integrate the SIR equations
    t = torch.arange(len(susceptible), dtype=torch.float)
    y = torch.tensor(sir_solution((S0, I0, R0), t, beta, gamma), dtype=torch.float)
    
    # Compare the model's output with the observed data
    with pyro.plate("data", len(susceptible)):
        pyro.sample("obs_S", dist.Normal(y[:, 0], 1000), obs=susceptible)
        pyro.sample("obs_I", dist.Normal(y[:, 1], 1000), obs=infected)
        pyro.sample("obs_R", dist.Normal(y[:, 2], 1000), obs=recovered)

In [7]:
# Load the data
us_daily_data = pd.read_csv("/home/matcraft/GitHub/DATA/COVID/us_covid19_daily.csv")

In [8]:
# Convert date columns to datetime format
us_daily_data['date'] = pd.to_datetime(us_daily_data['date'], format="%Y%m%d")

In [9]:
# Extract relevant columns for SIR modeling
sir_data = us_daily_data[['date', 'positive', 'death']].sort_values(by='date').reset_index(drop=True)

In [10]:
# Calculate the number of recovered individuals
recovery_delay = 14
sir_data['recovered'] = sir_data['positive'].shift(recovery_delay) - sir_data['death'].shift(recovery_delay)
sir_data['recovered'].fillna(0, inplace=True)

In [11]:
# For simplicity, we'll consider the entire US population as susceptible initially
us_population = 328_200_000  # US population estimate
sir_data['susceptible'] = us_population - sir_data['positive'] - sir_data['recovered']

In [12]:
# Convert data to PyTorch tensors
susceptible_t = torch.tensor(sir_data['susceptible'].values, dtype=torch.float)
infected_t = torch.tensor(sir_data['positive'].values, dtype=torch.float)
recovered_t = torch.tensor(sir_data['recovered'].values, dtype=torch.float)

In [13]:
# Run inference using Pyro
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

In [14]:
# Define the guide function
def guide(susceptible, infected, recovered):
    beta_loc = pyro.param('beta_loc', torch.tensor(0.5), constraint=dist.constraints.positive)
    beta_scale = pyro.param('beta_scale', torch.tensor(0.2), constraint=dist.constraints.positive)
    gamma_loc = pyro.param('gamma_loc', torch.tensor(0.5), constraint=dist.constraints.positive)
    gamma_scale = pyro.param('gamma_scale', torch.tensor(0.2), constraint=dist.constraints.positive)
    
    beta = pyro.sample("beta", dist.Normal(beta_loc, beta_scale))
    gamma = pyro.sample("gamma", dist.Normal(gamma_loc, gamma_scale))

In [15]:
def guide_modified(susceptible, infected, recovered):
    # Beta distribution parameters for beta
    alpha_beta = pyro.param("alpha_beta", torch.tensor(2.0), constraint=constraints.positive)
    beta_beta = pyro.param("beta_beta", torch.tensor(2.0), constraint=constraints.positive)
    
    # Beta distribution parameters for gamma
    alpha_gamma = pyro.param("alpha_gamma", torch.tensor(2.0), constraint=constraints.positive)
    beta_gamma = pyro.param("beta_gamma", torch.tensor(2.0), constraint=constraints.positive)
    
    beta = pyro.sample("beta", dist.Beta(alpha_beta, beta_beta))
    gamma = pyro.sample("gamma", dist.Beta(alpha_gamma, beta_gamma))

In [16]:
# Data conversion
susceptible_t = torch.tensor(susceptible).float()
infected_t = torch.tensor(infected).float()
recovered_t = torch.tensor(recovered).float()

NameError: name 'susceptible' is not defined

In [ ]:
# # # Setup the inference algorithm
# # pyro.clear_param_store()
# # # svi = SVI(model, guide, Adam({"lr": 0.001}), loss=Trace_ELBO())
# # optimizer = pyro.optim.Adam({"lr": 0.001})  # Reduced learning rate from 0.01 to 0.001
# # svi = pyro.infer.SVI(model, guide, optimizer, loss=pyro.infer.Trace_ELBO())

# # Resetting parameters
# pyro.clear_param_store()

# # Setting up SVI with modified model and guide
# svi = pyro.infer.SVI(sir_model_modified, guide_modified, optimizer, loss=pyro.infer.Trace_ELBO())
# Setup for SVI
pyro.clear_param_store()
optimizer = Adam({"lr": 0.01})
svi = SVI(sir_model_modified, guide_modified, optimizer, loss=Trace_ELBO())

In [ ]:
# # Train the model
# num_iterations = 5000
# for j in range(num_iterations):
#     loss = svi.step(susceptible_t, infected_t, recovered_t)
#     if j % 500 == 0:
#         print("[iteration %04d] loss: %.4f" % (j + 1, loss / len(susceptible_t)))
# Running SVI
losses = []
num_iterations = 5000
for j in range(num_iterations):
    loss = svi.step(susceptible_t, infected_t, recovered_t)
    losses.append(loss)
    if j % 500 == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss / len(susceptible_t)))

losses[-1]

In [ ]:
# Extract the inferred parameters
beta_estimated = pyro.param("beta_loc").item()
gamma_estimated = pyro.param("gamma_loc").item()

print(f"Estimated beta: {beta_estimated}")
print(f"Estimated gamma: {gamma_estimated}")